# Comparing alignment scores using Biopython

#### Using Covid variant RNA sequences

Here's a walkthrough for downloading sequences from the NIH, parsing the fasta files and then aligning ref. against test for covid variants. 

This was transcribed from a biopython tutorial from DataQuest.

In [1]:
import pandas as pd
from Bio import Entrez #entrez has an esearch function, which searches NIH database for a given sequence

In [2]:
Entrez.email = "None"

In [3]:
def download_sequence(id): #given a nucleotide accession, download sequence
    handle = Entrez.esearch(db="nucleotide", term=id, retmax='1') #search database, using the id (accession num.) 
    record = Entrez.read(handle) #generates a second id, to be used below
    handle = Entrez.efetch(db="nucleotide", id=record["IdList"][0], rettype='fasta', retmode='text') #the id from above is passed in from above
    return handle.read()

### **Choosing sequences to test**

**Reference:** NC_045512.2 (first isolated and sequenced strain from Wuhan)

**Test:** OM061695.1 (Delta, isolated 2021

In [4]:
sequences = ['NC_045512.2', 'OM061695.1']

### **Downloading sequences**
To loop through all of the sequences in the sequence list, at each iteration download from the NIH the sequences and store within the dictionary

In [5]:
sequence_data = {}
for sequence in sequences:
    sequence_data[sequence] = {"fasta": download_sequence(sequence)}

### **Parsing fasta files**
Now, the files are in a 'fasta' format. We'll parse it to something easier to work with. 

In [6]:
from Bio import SeqIO
import io


In [7]:
for k,v in sequence_data.items(): #loop through sequence data dict above. note: key = id, value = sequence (in fasta format)
    f = io.StringIO(v['fasta']) #create file object using fasta sequence. necessary to create a file object as biopython only works with files (this is essentially creating a 'fake' file in memory)
    sequence_data[k]["parsed"] = list(SeqIO.parse(f, "fasta"))[0] #parse above using biopython fasta parser. [0] is included as parsing will creating several objects, but we select the first
    

In the sequence data, there will now be a parsed field

In [8]:
sequence_data['NC_045512.2']['parsed']

SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA'), id='NC_045512.2', name='NC_045512.2', description='NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome', dbxrefs=[])

In [9]:
from Bio import Align
aligner = Align.PairwiseAligner()

In [10]:
score = aligner.score(sequence_data['NC_045512.2']['parsed'].seq, sequence_data['OM061695.1']['parsed'].seq)

In [11]:
score #to tell me how aligned the scores are, in relation to total number of nucleotides

29818.0

In [12]:
print(len(sequence_data['NC_045512.2']['parsed'].seq)) #length of the reference strain

29903


In [13]:
alignment_score = 29818.0 / 29903
alignment_score

0.9971574758385446

# Sequence mismatches
in order to obtain the points of mutations, we'll align the two sequences

In [14]:
reference = sequence_data['NC_045512.2']['parsed'].seq
delta = sequence_data['OM061695.1']['parsed'].seq

In [15]:
delta_alignments = aligner.align(reference, delta)

In [16]:
delta_alignment = delta_alignments[0]

In [17]:
delta_alignment.shape #the below output will tell us how many sequences were aligned, and the amount of nucleotides aligned

(2, 29943)

### How to interpret below output:
shows overlaps between sequences, e.g. 
- first item in first list (reference strain) is (0,209), (210,212)...
- second (delta) is (0,209), (209,211)



In [18]:
delta_alignment.aligned #this will show the points where sequences were aligned

(((0, 209),
  (210, 212),
  (212, 240),
  (241, 1047),
  (1048, 1273),
  (1274, 1276),
  (1276, 3036),
  (3037, 3038),
  (3038, 4180),
  (4181, 6401),
  (6402, 7123),
  (7123, 7124),
  (7125, 8985),
  (8986, 8989),
  (8989, 9052),
  (9053, 9054),
  (9054, 10028),
  (10028, 10029),
  (10030, 10506),
  (10507, 11116),
  (11117, 11200),
  (11201, 11331),
  (11332, 11333),
  (11333, 14407),
  (14408, 14409),
  (14409, 15450),
  (15450, 15451),
  (15452, 16465),
  (16466, 19219),
  (19220, 19221),
  (19221, 21617),
  (21618, 21986),
  (21987, 22028),
  (22029, 22030),
  (22033, 22035),
  (22037, 22916),
  (22917, 22918),
  (22918, 22994),
  (22995, 22996),
  (22996, 23402),
  (23403, 23603),
  (23604, 24409),
  (24410, 24411),
  (24411, 25087),
  (25088, 25090),
  (25090, 25351),
  (25352, 25353),
  (25353, 25468),
  (25469, 26423),
  (26424, 26425),
  (26425, 26766),
  (26767, 26768),
  (26768, 27526),
  (27527, 27637),
  (27637, 27639),
  (27640, 27751),
  (27752, 27873),
  (27874, 27876)

In [19]:
seq1_end = None
seq2_end = None
for alignments in zip(delta_alignment.aligned[0], delta_alignment.aligned[1]): #to take first element from both above tuple of tuples, and so on
    if seq1_end and seq2_end:
        seq1_mismatch = reference[seq1_end: alignments[0][0]] #gives us inverse of alignments i.e. all of the misalignments
        seq2_mismatch = delta[seq2_end: alignments[1][0]]
        print("1: {}".format(seq1_mismatch))
        print("2: {}".format(seq2_mismatch))
        
    seq1_end = alignments[0][1]
    seq2_end = alignments[1][1]

1: G
2: 
1: 
2: T
1: C
2: T
1: G
2: T
1: G
2: 
1: 
2: T
1: C
2: 
1: 
2: T
1: G
2: T
1: C
2: T
1: 
2: T
1: C
2: 
1: C
2: 
1: 
2: T
1: G
2: 
1: 
2: T
1: 
2: T
1: C
2: 
1: C
2: T
1: A
2: R
1: A
2: G
1: A
2: 
1: 
2: G
1: C
2: 
1: 
2: T
1: 
2: A
1: G
2: 
1: C
2: T
1: C
2: 
1: 
2: T
1: C
2: G
1: G
2: A
1: A
2: 
1: TTC
2: 
1: AG
2: 
1: T
2: 
1: 
2: G
1: C
2: 
1: 
2: A
1: A
2: G
1: C
2: G
1: G
2: 
1: 
2: A
1: G
2: 
1: 
2: T
1: G
2: 
1: 
2: T
1: C
2: T
1: T
2: 
1: 
2: C
1: T
2: 
1: 
2: C
1: C
2: T
1: 
2: C
1: T
2: 
1: C
2: T
1: C
2: 
1: 
2: T
1: G
2: 
1: TT
2: 
1: A
2: 
1: CT
2: 
1: A
2: 
1: A
2: G
1: 
2: T
1: G
2: 
1: 
2: T
1: G
2: 
1: G
2: T
1: G
2: T


**Above shows what the mismatches are, but not where**

# Finding mismatch location
logic, by checking lengths of mismatches

**Insertion**
- reference mismatch = 0, and delta mismatch = several

**deletion**
- as above, but reversed i.e. reference with mismatches

**substitution**
- matching in length

In [20]:
from IPython.display import HTML

In [21]:
def color_print(s, color='black'):
    return "<span style='color:{}'>{}</span>".format(color, s)

In [22]:
seq1_end = None
seq2_end = None
display_seq = []

for alignments in zip(delta_alignment.aligned[0], delta_alignment.aligned[1]): #to take first element from both above tuple of tuples, and so on
    if seq1_end and seq2_end:
        
        seq1_mismatch = reference[seq1_end: alignments[0][0]] #gives us inverse of alignments i.e. all of the misalignments
        seq2_mismatch = delta[seq2_end: alignments[1][0]]
        if len(seq2_mismatch) == 0:
            display_seq.append(color_print(seq1_mismatch, "red"))
        elif len(seq1_mismatch) == 0:
            display_seq.append(color_print(seq2_mismatch, "green"))
        else:
            display_seq.append(color_print(seq2_mismatch, "blue"))
        print("1: {}".format(seq1_mismatch))
        print("2: {}".format(seq2_mismatch))
    
    display_seq.append(reference[alignments[0][0]:alignments[0][1]])
    
    seq1_end = alignments[0][1]
    seq2_end = alignments[1][1]

1: G
2: 
1: 
2: T
1: C
2: T
1: G
2: T
1: G
2: 
1: 
2: T
1: C
2: 
1: 
2: T
1: G
2: T
1: C
2: T
1: 
2: T
1: C
2: 
1: C
2: 
1: 
2: T
1: G
2: 
1: 
2: T
1: 
2: T
1: C
2: 
1: C
2: T
1: A
2: R
1: A
2: G
1: A
2: 
1: 
2: G
1: C
2: 
1: 
2: T
1: 
2: A
1: G
2: 
1: C
2: T
1: C
2: 
1: 
2: T
1: C
2: G
1: G
2: A
1: A
2: 
1: TTC
2: 
1: AG
2: 
1: T
2: 
1: 
2: G
1: C
2: 
1: 
2: A
1: A
2: G
1: C
2: G
1: G
2: 
1: 
2: A
1: G
2: 
1: 
2: T
1: G
2: 
1: 
2: T
1: C
2: T
1: T
2: 
1: 
2: C
1: T
2: 
1: 
2: C
1: C
2: T
1: 
2: C
1: T
2: 
1: C
2: T
1: C
2: 
1: 
2: T
1: G
2: 
1: TT
2: 
1: A
2: 
1: CT
2: 
1: A
2: 
1: A
2: G
1: 
2: T
1: G
2: 
1: 
2: T
1: G
2: 
1: G
2: T
1: G
2: T


In [23]:
display_seq = [str(i) for i in display_seq]

In [24]:
display(HTML('<br>'.join(display_seq)))